# Домашнее задание № 7 

In [1]:
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

import os
import cv2
import skimage
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import RANSACRegressor
# from PCV.geometry import homography
# from PCV.localdescriptors import sift

# Поиск объектов в видеопотоке 

Задача - придумать и реализовать алгоритм поиска (обнаружения без классификации) движущихся объектов.

В качестве исходных данных приведена выборка с видеофайлами и аннотацией для каждого кадра файла. Аннотация задана в виде ограничивающих прямоугольников в формате ```(y1,x1,y2,x2)```, где
- ```(x1,y1)``` - верхний левый угол прямоугольника;
- ```(x2,y2)``` - нижний правый угол прямоугольника.

Ссылка на данные – https://disk.yandex.ru/d/RdjMDoQQO8Ngcw

В качестве обучающей можно брать любые видеофайлы. При этом должны быть отдельно выбраны тестовые данные, которые не будут использованы в создании решения. 

Видеофайл с результатами работы алгоритма должен быть прикреплен вместе с решением. Пример фрагмента видеофайла с результатом поиска объектов приведен ниже.

Исходный код может быть в формате ```.py``` или ```.ipynb```.

![annotation](annot_example.gif "annotation")

## Требования к результату
- поиск должен находить геометрические место объекта на видеоизображении. Геометрическое место задано ограничивающим прямоугольником (bounding box);
- продолжительность решения для любого одного видеофайла не должна превышать 10 минут;
- должна быть приведена оценка точности решения;
- привести демонстрацию результатов требется на одном из тестовых видеофайлов.

In [16]:
def create_video(save_dir, size, img_format='jpg', vido_format='avi'):
    out_name = Path(save_dir).parts[-1]

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(Path(save_dir) / Path(f'{out_name}.{vido_format}')),
                          fourcc, 20, tuple(size.astype(int)))

    for fname in tqdm(sorted(map(str, Path(save_dir).glob(f'*.{img_format}')))):
        imag = skimage.io.imread(fname)
        imag = cv2.cvtColor(imag, cv2.COLOR_RGB2BGR)
        out.write(imag)
    out.release()

In [3]:
def show_video(num=-1,path='./tmp/', sleep=0):
    cap = get_video(num if num >=0 else 0, path=path)
    get_video_details(cap)
    cv2.startWindowThread()
    while (cap.isOpened()):
        is_ok, frame = cap.read()
        if not is_ok:
            break
            
        if sleep > 0:
            time.sleep(sleep)

        cv2.imshow("sparse optical flow", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [23]:
def spec_points(image):
    """
    Функция для поиска особых точек и получения их дескрипторов 
    """
    
    hyp_params = dict(
        nfeatures = 200,
        nOctaveLayers = 10,
        contrastThreshold = 0.01,
        edgeThreshold = 6,
        sigma = 2.0)  # hyp params
    detector = cv2.SIFT_create(**hyp_params)

    keypoints, desc = detector.detectAndCompute(image.copy(), None)
    return keypoints, desc

In [24]:
def get_keyp_and_d(first, second):
    FLANN_INDEX_KDTREE = 2
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    ratio_thresh = 0.5
    keypoints1, desc1 = spec_points(first)
    keypoints2, desc2 = spec_points(second)
    
    matches = flann.knnMatch(desc1, desc2, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_thresh * n.distance:
            good_matches.append(m)
    
    return keypoints1, keypoints2, good_matches

In [25]:
def get_M(first, second):
    h, w = first.shape
    
    
    keypoints1, keypoints2, good_matches = get_keyp_and_d(first, second)
    
    pts1 = []
    pts2 = []   
    
    for good_matche in good_matches:
        pts1.append(keypoints1[good_matche.queryIdx].pt)
        pts2.append(keypoints2[good_matche.trainIdx].pt)
    
    pts1 = np.array(pts1).astype(np.float32)
    pts2 = np.array(pts2).astype(np.float32)
    
#     print('pts1', pts1)
#     print('pts2', pts2)
    
    model = RANSACRegressor()
    model.fit(pts1, pts2)

    points1 = np.array([[0,0], [h,0], [0,w], [h,w]]).astype(np.float32)
    points2 = model.predict(points1).astype(np.float32)
    M = cv2.getPerspectiveTransform(points1, points2)
    return M
    

In [26]:
def get_trajectory(cap, max_idx=-1):
    cnt, w, h, fps = get_video_details(cap)
    cnt = int(cnt)
    color = (0, 255, 0)
    feature_params = dict(maxCorners=300, qualityLevel=0.2, minDistance=2, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
    is_ok, first_frame = cap.read()
    if not is_ok:
        print("ERR: Can't read")
        return
#     plt.imshow(first_frame)
#     plt.show()
    
    prev_gray = cv2.cvtColor(first_frame.copy(), cv2.COLOR_BGR2GRAY)
#     prev = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
    mask = np.zeros_like(first_frame)
    idx = 0
    for idx in tqdm(range(cnt-1)):
        if not cap.isOpened():
            break
#         print("idx", idx)
        if max_idx > 0 and idx > max_idx:
            break
        is_ok, frame = cap.read()
#         if not is_ok or idx == 10:
#             break
        if not is_ok:
            break
        gray = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2GRAY)
        mask = np.zeros_like(frame)
        
#         print('prob:', np.sum((prev_gray - gray)**2))
        M = get_M(prev_gray.copy(), gray.copy())
#         if np.sum(np.abs(M - np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]]))) < 0.1:
#             print('bad M')
#             break
#         print('M',M)
        trans_prev_gray = cv2.warpPerspective(prev_gray.copy(), M, (prev_gray.shape[1], prev_gray.shape[0]))
        
#         plt.imshow(trans_prev_gray, cmap='gray')
#         plt.show()

        prev = cv2.goodFeaturesToTrack(trans_prev_gray, mask=None, **feature_params)
    
        trans_gray = gray*(trans_prev_gray!=0).astype(np.uint8)
        nextp, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, trans_gray, prev, None, **lk_params)

        good_old = prev[status == 1].astype(int)
        good_new = nextp[status == 1].astype(int)

        # Draws the optical flow tracks
        for i, (new, old) in enumerate(zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (a, b), (c, d), color, 2)
            frame = cv2.circle(frame, (a, b), 3, color, -1)

        output = cv2.add(frame, mask)
        prev_gray = gray.copy()
        # Updates previous good feature points
        prev = good_new.reshape(-1, 1, 2)

        cv2.imwrite(f'tmp/frame_{str(idx).zfill(4)}.jpg', output)
#         idx += 1
        
        # Opens a new window and displays the output frame
#         cv2.imshow("sparse optical flow", output)
        
#         idx += 1
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

    # The following frees up resources and closes all windows
    cap.release()

In [27]:
cap = get_video(1)
get_trajectory(cap)
create_video('tmp', size=np.array((1920.0 , 1080.0)), vido_format='mp4')


['Clip_1.mov', 'Clip_10.mov', 'Clip_11.mov', 'Clip_2.mov', 'Clip_3.mov', 'Clip_37.mov', 'Clip_4.mov', 'Clip_5.mov', 'Clip_6.mov', 'Clip_7.mov', 'Clip_8.mov', 'Clip_9.mov']
./Videos/Videos/Clip_10.mov
1800.0 1920.0 1080.0 29.97


  0%|          | 0/1799 [00:00<?, ?it/s]

  0%|          | 0/1799 [00:00<?, ?it/s]

In [28]:
show_video(sleep=0.1)

['tmp.mp4']
./tmp/tmp.mp4
1799.0 1920.0 1080.0 20.0


In [8]:
def get_video_details(cap):
    cnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(cnt, w, h, fps)
    return cnt, w, h, fps

In [9]:
def get_video(num, path='./Videos/Videos/'):
    """
    Функция достающая видео
    num - номер видео (если num == -1, то достаются все видео)
    
    """
    all_names = os.listdir(path)
    names = []
    for name in all_names:
        if name.endswith(".mov") or name.endswith(".mp4"):
            names.append(name)
    print(names)
    if num >= 0:
        name_vid = names[num]
        print(path+name_vid)
        return cv2.VideoCapture(path+name_vid)
    
    videos = []
    for name_vid in names:
        videos.append(cv2.VideoCapture(path+name_vid))
    return videos
    

In [11]:
show_video(0, './Videos/Videos/', 1)

['Clip_1.mov', 'Clip_10.mov', 'Clip_11.mov', 'Clip_2.mov', 'Clip_3.mov', 'Clip_37.mov', 'Clip_4.mov', 'Clip_5.mov', 'Clip_6.mov', 'Clip_7.mov', 'Clip_8.mov', 'Clip_9.mov']
./Videos/Videos/Clip_1.mov
309.0 1920.0 1080.0 29.97


KeyboardInterrupt: 

In [14]:
show_video(sleep=0.2)

['tmp.mp4']
./tmp/tmp.mp4
308.0 1920.0 1080.0 20.0


In [17]:
create_video('tmp', size=np.array((1920.0 , 1080.0)), vido_format='mov')
show_video(sleep=0.2)

  0%|          | 0/308 [00:00<?, ?it/s]

['tmp.mov']
./tmp/tmp.mov
308.0 1920.0 1080.0 20.0
